# Julia Workshop: Optimization and Solvers

## @ CEF 2017

**Authors**: Chase Coleman and Spencer Lyon

**Date**: 27 June 2017


## Goal

A few different packages used for optimization or non-linear equation solving.


## QuantEcon

The QuantEcon library has a few simple optimizers and solvers. See the economics [notebook](economics.ipynb) for more information.

In [1]:
# Pkg.add("QuantEcon")

## Optim.jl

Package for optimizing written in pure Julia

In [2]:
# Pkg.add("Optim")

In [3]:
using Optim

In [4]:
rosenbrock(x) = (1.0 - x[1])^2 + 100.0*(x[2] - x[1]^2)^2

rosenbrock (generic function with 1 method)

### Optimizing without gradient

In [5]:
optimize(rosenbrock, zeros(2), NelderMead())

Results of Optimization Algorithm
 * Algorithm: Nelder-Mead
 * Starting Point: [0.0,0.0]
 * Minimizer: [0.9999710322210338,0.9999438685860869]
 * Minimum: 1.164323e-09
 * Iterations: 74
 * Convergence: true
   *  √(Σ(yᵢ-ȳ)²)/n < 1.0e-08: true
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 108

In [6]:
optimize(rosenbrock, zeros(2), BFGS())

Results of Optimization Algorithm
 * Algorithm: BFGS
 * Starting Point: [0.0,0.0]
 * Minimizer: [0.9999999926033423,0.9999999852005353]
 * Minimum: 5.471433e-17
 * Iterations: 16
 * Convergence: true
   * |x - x'| < 1.0e-32: false
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: false
   * |g(x)| < 1.0e-08: true
   * f(x) > f(x'): false
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 69
 * Gradient Calls: 69

### Optimizing with gradient

In [7]:
function rosenbrock_grad!(x::Vector, grad::Vector)
    grad[1] = -2.0*(1.0 - x[1]) - 400.0*(x[2] - x[1]^2)*x[1]
    grad[2] = 200.0*(x[2] - x[1]^2)
end

rosenbrock_grad! (generic function with 1 method)

In [8]:
optimize(rosenbrock, rosenbrock_grad!, zeros(2), LBFGS(),
         Optim.Options(x_tol=1e-10, f_tol=1e-9, iterations=25000,
                       allow_f_increases=true))

Results of Optimization Algorithm
 * Algorithm: L-BFGS
 * Starting Point: [0.0,0.0]
 * Minimizer: [1.0000000000000007,1.000000000000001]
 * Minimum: 5.374115e-30
 * Iterations: 21
 * Convergence: true
   * |x - x'| < 1.0e-10: false
   * |f(x) - f(x')| / |f(x)| < 1.0e-09: true
   * |g(x)| < 1.0e-08: true
   * f(x) > f(x'): false
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 90
 * Gradient Calls: 90

In [9]:
optimize(rosenbrock, rosenbrock_grad!, zeros(2), GradientDescent(),
         Optim.Options(x_tol=1e-10, f_tol=1e-9, iterations=25000,
                       allow_f_increases=true))

Results of Optimization Algorithm
 * Algorithm: Gradient Descent
 * Starting Point: [0.0,0.0]
 * Minimizer: [0.9999999572646519,0.9999999142802773]
 * Minimum: 1.832511e-15
 * Iterations: 18460
 * Convergence: true
   * |x - x'| < 1.0e-10: true
   * |f(x) - f(x')| / |f(x)| < 1.0e-09: false
   * |g(x)| < 1.0e-08: false
   * f(x) > f(x'): false
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 64642
 * Gradient Calls: 64642

### Optimizing with Hessian

In [10]:
function rosenbrock_hess!(x::Vector, hess::Matrix)
    hess[1, 1] = 2.0 - 400.0 * x[2] + 1200.0*x[1]^2
    hess[1, 2] = -400.0 * x[1]
    hess[2, 1] = -400.0 * x[1]
    hess[2, 2] = 200.0
end

rosenbrock_hess! (generic function with 1 method)

In [11]:
optimize(rosenbrock, rosenbrock_grad!, rosenbrock_hess!, zeros(2), Newton())

Results of Optimization Algorithm
 * Algorithm: Newton's Method
 * Starting Point: [0.0,0.0]
 * Minimizer: [0.9999999999999994,0.9999999999999989]
 * Minimum: 3.081488e-31
 * Iterations: 14
 * Convergence: true
   * |x - x'| < 1.0e-32: false
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: false
   * |g(x)| < 1.0e-08: true
   * f(x) > f(x'): false
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 58
 * Gradient Calls: 58

## NLopt

Julia wrapper to high quality C library.

This library has _lots_ of options for algorithms. See [list](http://ab-initio.mit.edu/wiki/index.php/NLopt_Algorithms) of algorithms

In [12]:
# Pkg.add("NLopt")

In [13]:
using NLopt

In [14]:
function rosenbrock_nlopt(x::Vector, grad::Vector)
    if length(grad) > 0
        rosenbrock_grad!(x, grad)
    end

    return rosenbrock(x)
end

rosenbrock_nlopt (generic function with 1 method)

### A non-gradient and gradient based method

In [31]:
opt_LBFGS = Opt(:LD_LBFGS, 2)

min_objective!(opt_LBFGS, rosenbrock_nlopt)
xtol_rel!(opt_LBFGS, 1e-10)
ftol_rel!(opt_LBFGS, 1e-9)

NLopt.optimize(opt_LBFGS, zeros(2))

(3.251384063527492e-21,[1.0,1.0],:SUCCESS)

In [24]:
opt_PRAXIS = Opt(:LN_PRAXIS, 2)

min_objective!(opt_PRAXIS, rosenbrock_nlopt)

NLopt.optimize(opt_PRAXIS, zeros(2))

(1.5954711808094964e-28,[1.0,1.0],:SUCCESS)

### A constrained optimization problem

\begin{align*}
  \min_{x_1, x_2} &\sqrt{x_2} \\
  &\text{subject to } \\
  &x_2 \geq 0 \\
  &x_2 \geq (2 x_1)^3  \\
  &x_2 \geq (-x_1 + 1)^3
\end{align*}

In [17]:
function myfunc(x::Vector, grad::Vector)
    if length(grad) > 0
        grad[1] = 0
        grad[2] = 0.5/sqrt(x[2])
    end

    return sqrt(x[1] + x[2])
end

function myconstraint(x::Vector, grad::Vector, a, b)
    if length(grad) > 0
        grad[1] = 3a * (a*x[1] + b)^2
        grad[2] = -1
    end

    return (a*x[1] + b)^3 - x[2]
end

myconstraint (generic function with 1 method)

In [18]:
opt = Opt(:LD_MMA, 2)

lower_bounds!(opt, [-Inf, 0.])
xtol_rel!(opt, 1e-6)

min_objective!(opt, myfunc)

inequality_constraint!(opt, (x,g) -> myconstraint(x, g, 2.0, 0.0), 1e-8)
inequality_constraint!(opt, (x,g) -> myconstraint(x, g, -1.0 ,1.0), 1e-8)

(minf, minx, ret) = NLopt.optimize(opt, [1.234, 5.678])

println("got $minf at $minx (returned $ret)")

got 0.7934920514599207 at [0.333333,0.296296] (returned XTOL_REACHED)


## NLsolve

Julia package written to solve systems of non-linear equations.

In [19]:
# Pkg.add("NLsolve")

In [20]:
using NLsolve

In [21]:
function f!(xy::Vector, fxy::Vector)
    # Pull out arguments
    x, y = xy

    # Fill fxy
    fxy[1] = x^2 - sin(y)
    fxy[2] = y^2 - cos(x)
end

function g!(xy::Vector, jacxy::Matrix)
    x, y = xy
    # Fill with derivatives of first function
    jacxy[1, 1] = 2*x
    jacxy[1, 2] = -cos(y)

    # Fill off-diagonal
    jacxy[2, 1] = sin(x)
    jacxy[2, 2] = 2*y
end

res = nlsolve(f!, g!, [0.0, 0.0], ftol=1e-10)

Results of Nonlinear Solver Algorithm
 * Algorithm: Trust-region with dogleg and autoscaling
 * Starting Point: [0.0,0.0]
 * Zero: [0.0,0.0]
 * Inf-norm of residuals: 1.000000
 * Iterations: 1000
 * Convergence: false
   * |x - x'| < 0.0e+00: false
   * |f(x)| < 1.0e-10: false
 * Function Calls (f): 1001
 * Jacobian Calls (df/dx): 1